In [9]:
import pandas as pd
import sklearn.metrics
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import VotingClassifier

In [10]:
df = pd.read_csv('./../clean_data.csv')

In [11]:
labels = df.columns[1:]
x=df.loc[:,labels]
y=df.loc[:,'churn']

xtrain, xtest, ytrain, ytest = train_test_split(x,y,train_size=0.8)

In [12]:
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(xtrain, ytrain)

# Display class distribution after oversampling
print("\nClass distribution after oversampling:")
print(pd.Series(y_resampled).value_counts())

# Apply RandomUnderSampler to address class imbalance
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(xtrain, ytrain)

# Display class distribution after undersampling
print("\nClass distribution after undersampling:")
print(pd.Series(y_resampled).value_counts())


Class distribution after oversampling:
No     3256
Yes    3256
Name: churn, dtype: int64

Class distribution after undersampling:
No     534
Yes    534
Name: churn, dtype: int64


In [13]:
params = {
    'hidden_layer_sizes': [(100,), (50, 50), (25, 25, 25)],
    'activation': ['relu', 'tanh', 'sigmoid'],
    'solver': ['adam', 'sgd', 'lbfgs'],
    'learning_rate': ['constant', 'adaptive'],
    'alpha': [0.0001, 0.001, 0.01],
    'batch_size': [16, 32, 64],
    'max_iter': [9999999999]
    }

clf_accuracy = GridSearchCV(estimator=MLPClassifier(),param_grid=params,cv=5,n_jobs=5,verbose=1,scoring='accuracy')
clf_accuracy.fit(xtrain,ytrain)

print(clf_accuracy.best_params_)
model_accuracy = MLPClassifier(**clf_accuracy.best_params_) 

f1 = sklearn.metrics.make_scorer(sklearn.metrics.f1_score, average='micro')

clf_f1 = GridSearchCV(estimator=MLPClassifier(),param_grid=params,cv=5,n_jobs=5,verbose=1,scoring=f1)
clf_f1.fit(xtrain,ytrain)

print(clf_f1.best_params_)
modelf1 = MLPClassifier(n_neighbors=clf_f1.best_params_['n_neighbors'], weights=clf_f1.best_params_['weights'], algorithm=clf_f1.best_params_['algorithm'], p=clf_f1.best_params_['p']) 

combi_model = VotingClassifier(estimators=[('acc', model_accuracy), ('f1', modelf1)], voting='soft')

Fitting 5 folds for each of 486 candidates, totalling 2430 fits


KeyboardInterrupt: 

In [ ]:
cv_strategy = KFold(n_splits=5, shuffle=True, random_state=42)
cross_val_scores = cross_val_score(model_accuracy, x, y, cv=cv_strategy, scoring='accuracy')

print("Cross-Validation Scores:", cross_val_scores)
print("Mean Accuracy:", cross_val_scores.mean())
print("Standard Deviation:", cross_val_scores.std())

/usr/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[0.93037975 0.94303797 0.94198312 0.93980993 0.94508976]


/usr/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
model_accuracy.fit(X_resampled,y_resampled)

yhat = model_accuracy.predict(xtest)

acc = sklearn.metrics.accuracy_score(ytest,yhat)
recall = sklearn.metrics.recall_score(ytest,yhat,pos_label='Yes')
prec = sklearn.metrics.precision_score(ytest,yhat,pos_label='Yes')
f1 = sklearn.metrics.f1_score(ytest,yhat,pos_label='Yes')
conf_matrix =  sklearn.metrics.confusion_matrix(ytest,yhat)
print('balanced model')
print(f"accuracy: {acc}")
print(f"recall: {recall}")
print(f"precision: {prec}")
print(f"f1: {f1}")
print(f"confusion matrix:\n{conf_matrix}\n")

modelf1.fit(X_resampled,y_resampled)

yhat = modelf1.predict(xtest)

acc = sklearn.metrics.accuracy_score(ytest,yhat)
recall = sklearn.metrics.recall_score(ytest,yhat,pos_label='Yes')
prec = sklearn.metrics.precision_score(ytest,yhat,pos_label='Yes')
f1 = sklearn.metrics.f1_score(ytest,yhat,pos_label='Yes')
conf_matrix =  sklearn.metrics.confusion_matrix(ytest,yhat)
print('f1 model')
print(f"accuracy: {acc}")
print(f"recall: {recall}")
print(f"precision: {prec}")
print(f"f1: {f1}")
print(f"confusion matrix:\n{conf_matrix}\n")

combi_model.fit(X_resampled,y_resampled)

yhat = combi_model.predict(xtest)

acc = sklearn.metrics.accuracy_score(ytest,yhat)
recall = sklearn.metrics.recall_score(ytest,yhat,pos_label='Yes')
prec = sklearn.metrics.precision_score(ytest,yhat,pos_label='Yes')
f1 = sklearn.metrics.f1_score(ytest,yhat,pos_label='Yes')
conf_matrix =  sklearn.metrics.confusion_matrix(ytest,yhat)
print('combination model')
print(f"accuracy: {acc}")
print(f"recall: {recall}")
print(f"precision: {prec}")
print(f"f1: {f1}")
print(f"confusion matrix:\n{conf_matrix}\n")

0.9556962025316456
0.7520661157024794
0.883495145631068
0.8125000000000001
[[815  12]
 [ 30  91]]


/usr/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
